# Map Reduce Oefening - Schaken

**Deadline:**

In deze oefening gaan we werken met een dataset dat informatie bevat over een groot aantal schaakspelletjes.
Deze dataset kan [hier](https://www.kaggle.com/datasnaek/chess) gevonden worden.
Om te beginnen, download deze file en upload hem naar je distributed file systeem onder het path: **Oefingen/Mapreduce**.
Voorzie ook code die deze folder reset naar een lege folder om geen naam-conflicten te hebben.

In [ ]:
# download dataset
import pydoop.hdfs as hdfs
import opendatasets as od

od. download('https://www.kaggle.com/datasnaek/chess')

# reset folder to clean state
localFS = hdfs.hdfs(host='')
clientFS = hdfs.hdfs(host='localhost', 9000)

path = '/user/bigdata/oefeningen/mapreduce'
if not clientFS.exists(path):
    clientFS.create_directory(path)
    
for f in clientFS.list_directory(path):
    if not f['name'].endswith('games.csv'):
        clientFS.delete(f['name'], True)   # True omdat het ook mappen zou verwijderen

# upload de dataset naar hadoop
localFS.copy('chess/games.csv', client, path + '/games.csv')

# Map Reduce applicaties

Nu dat de dataset geupload is kunnen we map-reduce applicaties schrijven om deze dataset te verwerken. 
Deze oefeningen bestaan steeds uit twee cellen:
* Een eerste cel met de nodige python code voor de map-reduce applicatie. Sla deze code op in een file met het correcte oefeningennummer (bvb: oefening_1.py) door gebruik te maken van de "%%file" tag in de notebookcellen.
* Een tweede cel die de map-reduce applicatie uitvoert op de cluster en het correcte bestand uitleest. Commando's uitvoeren naar de commandline/terminal gebeurt door een uitroepingsteken vooraan te plaatsen (!pydoop submit ...)

## Aantal rijen

De eerste gevraagde map-reduce applicatie telt het aantal rijen.
Schrijf deze nu hieronder en beantwoord de vragen na de code:

In [ ]:
# map-reduce applicatie voor het aantal rijen te tellen
%%file wordcount_pydoop.py
import pydoop.mapreduce.api as api
import pydoop.mapreduce.pipes as pipes

class Mapper(api.Mapper):
    def map(self, context):
        # if !context.value.startswith('id,'):
        if context.key != 0:
            context.emit('aantal rijen', 1)
    
class Reducer(api.Reducer):
    def reduce(self, context):
        context.emit(context.key, sum(context.values))
        
FACTORY = pipes.Factory(Mapper, reducer_class=Reducer)

def main():
    pipes.run_task(FACTORY)

if __name__ == "__main__":
    main()

In [ ]:
# commando voor uitvoeren van de applicatie

**Vragen:**

- Hoeveel rijen heb je gevonden in de dataset?
- Hoeveel spelletjes zitten er echt in de dataset (kijk hiervoor naar de link van de dataset)?
- Zijn deze waarden gelijk? Indien nee, wat is de oorzaak van het verschil? Hoe zou je dit kunnen oplossen?

**Antwoord:**

Schrijf hier je antwoord...

Indien je waarden niet overeenkomen, maak een kopie van de vorige applicatie en pas het aan zodat het aantal spelletjes in de dataset correct geteld wordt.

In [ ]:
# aangepast map-reduce applicatie om het correcte aantal te bekomen.

In [ ]:
# commando voor uitvoeren van de applicatie

## Aantal kolommen

Naast het aantal rijen kunnen we ook het aantal kolommen bepalen. Aangezien het mogelijk zou zijn dat verschillende rijen een verschillend aantal kolommen hebben is het nodig om hier het maximum aantal kolommen in de verschillende rijen te bepalen.

In [ ]:
# map-reduce applicatie om het aantal kolommen te berekenen

In [ ]:
# commando voor uitvoeren van de applicatie

**Vragen:**

- Hoeveel kolommen heb je gevonden in de dataset?
- Hoeveel kolommen zitten er echt in de dataset (kijk hiervoor naar de link van de dataset)?
- Zijn deze waarden gelijk? Indien nee, wat is de oorzaak van het verschil? Hoe zou je dit kunnen oplossen?

**Antwoord:**

Schrijf hier je antwoord...

Indien je waarden niet overeenkomen, maak een kopie van de vorige applicatie en pas het aan zodat het aantal spelletjes in de dataset correct geteld wordt.

In [ ]:
# aangepast map-reduce applicatie om het correcte aantal te bekomen.

## Aantal uitkomsten

Er zijn vier verschillende uitkomsten mogelijk bij een schaakspel: mate (schaakmat), resign (opgave), draw(gelijkspel), out of time (tijd overschreden).
Maak nu een map-reduce applicatie dat telt hoe vaak elk van deze uitkomsten plaatsvindt

In [ ]:
# map-reduce applicatie om het aantal keer dat de verschillende uitkomsten voorkomen te berekenen

In [ ]:
# commando voor uitvoeren van de applicatie

**Vragen:**

* Hoeveel komt elke waarde voor?

**Antwoord:**

Schrijf hier je antwoord...

## Welke kleur wint het meest

Schrijf nu een map-reduce applicatie dat telt hoevaak elk kleur wint. Zorg er ook voor dat je naast de totaal aantal gewonnen spelletjes, ook enkel de spelletjes telt waar de rating van de spelers niet sterk verschilt (max 100 verschil).

In [ ]:
# map-reduce applicatie om de winstkansen van een kleur te bepalen.

In [ ]:
# commando voor uitvoeren van de applicatie

**Vragen:**

* Welke kleur heeft het meeste gewonnen?
* Is hetzelfde vast te stellen wanneer we enkel kijken naar gelijkwaardige spelers? (Bereken hiervoor de percentages!)

**Antwoord:**

Schrijf hier je antwoord...

## Verschillende openingen

Een belangrijk onderdeel van schaakspelletjes is de openingset die gebruikt wordt. Schrijf nu een map-reduce applicatie dat telt hoevaak elke openingsset resulteert in een winst of verlies voor de witte speler. 
Emit hierbij in de mapping fase enkel key-value pairs waar de keys de namen van de openingszetten zijn.
Filteren tussen winst en verlies moet dan in de reduce fase gebeuren.

In [ ]:
# winst of verlies van openingszetten
%%file openingen.py
import pydoop.mapreduce.api as api
import pydoop.mapreduce.pipes as pipes

class Mapper(api.Mapper):
    def map(self, context):
        if context.key != 0:
            cols = context.value.split(',')
            context.emit(cols[-2], cols[6])
    
class Reducer(api.Reducer):
    def reduce(self, context):
        # dit gaat per opening gebeuren -> dus je krijgt een lijstje met wit, zwart, zwart, wit, wit, ...
        wit = 0
        zwart = 0
        for winner in context.values:
            if winner == 'white':
                wit += 1
            else:
                zwart += 1
        context.emit(context.key + '-winst wit', wit)
        context.emit(context.key + '-winst zwart', zwart)
        
FACTORY = pipes.Factory(Mapper, reducer_class=Reducer)

def main():
    pipes.run_task(FACTORY)

if __name__ == "__main__":
    main()

In [ ]:
# commando voor uitvoeren van de applicatie

De output van bovenstaande applicatie zou een groot aantal rijen moeten bevatten. Download het bestand en bekijk het aantal rijen in dit bestand. 

**Hoeveel verschillende openingszetten zijn er aanwezig in de dataset?**

Schrijf hier je antwoord

## Verificatie van de kolom met het aantal zetten

In de dataset zijn twee manieren aanwezig om het aantal zetten in een schaakspel te bekomen.
* Er is een turns kolom dat het totaal aantal beurten vast heeft
* Er is ook een moves kolom dat alle uitgevoerde zetten bevat (gescheidden door een spatie)

Kijk of deze waarden voor alle rijen overeenkomt en tel de volgende waarden:
* Totaal aantal schaakspelletjes
* Aantal waar de kolommen overeenkomen
* Aantal waar de kolommen niet overeenkomen.

Zorg ervoor dat deze drie waarden aanwezig zijn in het uiteindelijke resultaat.

In [ ]:
# verificatie "turns" kolom

In [ ]:
# commando voor uitvoeren van de applicatie

**Plaats hieronder de bekomen output voor dit commando:**

Vervang dit door je output.

## Aantal verschillende spelers en aantal verschillende openingen

Schrijf nu een map-reduce applicatie dat zowel het aantal verschillende spelers als het aantal verschillende openingen in de dataset telt. Tip: een set in python bevat geen duplicaten

In [ ]:
# aantal spelers en openingen
%%file openingen.py
import pydoop.mapreduce.api as api
import pydoop.mapreduce.pipes as pipes

class Mapper(api.Mapper):
    def map(self, context):
        if context.key != 0:
            cols = context.value.split(',')
            context.emit('openingen', cols[-2]) # openingen
            context.emit('speler', cols[8]) # emit witte speler  w1, w2, w3
            context.emit('speler', cols[10]) # emit zwarte speler (combineer met de witte)    z1, z2, z3
    
class Reducer(api.Reducer):
    def reduce(self, context):
        # context.values van speler is w1, z1, w2, z2, w3, z3, ...
        unieke = set(context.values)
        context.emit(context.key, len(unieke))
        
FACTORY = pipes.Factory(Mapper, reducer_class=Reducer)

def main():
    pipes.run_task(FACTORY)

if __name__ == "__main__":
    main()

In [ ]:
# commando voor uitvoeren van de applicatie

**Plaats hieronder de bekomen output voor dit commando:**

Vervang dit door je output.

## verscheidene statistieken in 1 applicatie

Maak nu een map-reduce applicatie dat de volgende zaken in 1 keer berekend:
* Het aantal minuten dat een schaakspel geduurd heeft (de tijden aanwezig in de dataset zijn in [Unix-format](https://en.wikipedia.org/wiki/Unix_time))
* Het minimum, maximum en gemiddeld aantal beurten
* Percentage rated
* Het minimum, maximum en gemiddeld aantal keer dat er schaak voorkwam in een spel dat eindige in schaakmat. (Schaak wordt aangegeven door een "+" in de moves kolom.)

In [ ]:
# verscheidene statistieken
# aantal spelers en openingen
%%file openingen.py
import pydoop.mapreduce.api as api
import pydoop.mapreduce.pipes as pipes
import statistics

class Mapper(api.Mapper):
    def map(self, context):
        if context.key != 0:
            cols = context.value.split(',')
            
            context.emit('aantal minuten', float(cols[3]) - float(cols[2]) /1000 / 60)
            
            context.emit('aantal beurten', int(cols[4]))
            
            if cols[1] == 'true':
                context.emit('rated', 1)
            else:
                context.emit('rated', 0)
    
class Reducer(api.Reducer):
    def reduce(self, context):
        values = list(context.values)   # maak er een lijst van ipv een iterator, deze kan meerdere keren hergebruikt worden
        if context.key == 'aantal minuten':
            context.emit('mediaan duurtijd', statistics.median(context.values))
        elif context.key == 'aantal beurten':
            # andere oplossing dan values om zelf een for lus te schrijven
            context.emit(context.key + '-min', min(values))
            context.emit(context.key + '-max', max(values))
            context.emit(context.key + '-avg', sum(values) / len(values))
        elif context.key = 'rated':
            context.emit('perc rated', sum(values) / len(values))
        
FACTORY = pipes.Factory(Mapper, reducer_class=Reducer)

def main():
    pipes.run_task(FACTORY)

if __name__ == "__main__":
    main()

In [ ]:
# commando voor uitvoeren van de applicatie

**Plaats hieronder de bekomen output voor dit commando:**

Vervang dit door je output.